1. single inheritance: class subClass(baseClass). 1) Subclass will want to initialize base classes.  2) Base class initializer will only be automatically called if subclass initializer is undefined. Python treats __init__ like any other methods, base class __init__() will not be called if overriden.
2. multiple inheritance: class subClass(base1, base2, ..., basen). Subclass inherits all bases. Without conflict, names resolve in the obvious way; MRO (method resolution order) determines name lookup in all cases.
3. If __init__() is not defined in a subclass, the __init__ of the first baseclass will be called automatically.
4. C3 is an algo to calculate MRO in python. 1) Subclasses come before base classes. 2) Base class order from class definition is preserved. 3) First two qualities are preserved no matter what you start in the inheritance graph.
5. Class-bound super proxies: super(baseClass, derivedClass). 1) Python finds MRO for the derived class; 2) it then finds base class in that MRO; 3) it takes everything after the base class and finds the first class in that sequence with a matching method name.
6. Instance-bound super proxies: super(class, instance_of_class). 1) Python finds MRO for the type of the second argument; 2) it finds the location of the first argument in the MRO; 3) it uses everything after that for resolving methods.
7. Calling super() without argument: super() = super(class-of-method, self) for instance method; super() = super(class-of-method, class) for class method.

In [1]:
class Base:
    def __init__(self):
        print ('Base initializer')
    def f(self):
        print ('base.f()')
        
class Sub(Base):
    pass

In [2]:
s = Sub()

Base initializer


In [3]:
class Base:
    def __init__(self):
        print ('Base initializer')
    def f(self):
        print ('base.f()')
        
class Sub(Base):
    def __init__(self):
        super().__init__() # explicitly called
        print ('Sub initializer')
    def f(self):
        print ('sub.f()')

In [4]:
a = Sub()

Base initializer
Sub initializer


In [5]:
class SimpleList:
    def __init__(self, items):
        self._items = list(items)
    
    def add(self, item):
        self._items.append(item)
        
    def __getitem__(self, index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return 'SimpleList({!r})'.format(self._items)

In [6]:
class SortedList(SimpleList):
    def __init__(self, items):
        super().__init__(items)
        self.sort()
    def add(self, item):
        super().add(item)
        self.sort()
    def __repr__(self):
        return 'SortedList({!r})'.format(list(self))

In [9]:
sl = SortedList([2, -1, -10, 1])
print(sl)

sl.add(0)
print(sl)

SortedList([-10, -1, 1, 2])
SortedList([-10, -1, 0, 1, 2])


In [7]:
# determines if one type is a subclass of another
issubclass(SortedList, SimpleList)

True

In [11]:
# to determine if an object is of a specified type
isinstance(sl, SortedList)

True

In [14]:
isinstance([], (float, list, str)) # if the first argument is of any type of the second argument 

True

In [16]:
class IntList(SimpleList):
    def __init__(self, items):
        for x in items:
            self._validate(x)
        super().__init__(items)
    
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only supports integer values')
            
    def add (self, item):
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return 'IntList({!r})'.format(list(self))

In [17]:
class SortedIntList(IntList, SortedList): # multiple inheritance
    def __repr__(self):
        return 'SortedIntList({!r})'.format(list(self))

In [18]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

In [19]:
TypeError.__bases__

(Exception,)

In [20]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [21]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

In [22]:
class A:
    pass
class B(A):
    pass
class C(A):
    pass
class D(B, A, C): # not all inheritance declarations will be allowed
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

In [30]:
float.mro()

[float, object]

In [31]:
object.mro()

[object]

In [32]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

class-bound super proxies

In [33]:
super(SortedList, SortedIntList).add

<function __main__.SimpleList.add(self, item)>

In [34]:
super(SortedIntList, SortedIntList)._validate('hello')

TypeError: IntList only supports integer values

instance-bound super proxies

In [35]:
sil = SortedIntList([5, 10, 15])
super(SortedList, sil).add(6)
print(sil)

super(SortedList, sil).add('hello')
print(sil)

SortedIntList([5, 10, 15, 6])
SortedIntList([5, 10, 15, 6, 'hello'])


In [36]:
import inspect
inspect.getmembers(object)

[('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dir__', <method '__dir__' of 'object' objects>),
 ('__doc__', 'The most base type'),
 ('__eq__', <slot wrapper '__eq__' of 'object' objects>),
 ('__format__', <method '__format__' of 'object' objects>),
 ('__ge__', <slot wrapper '__ge__' of 'object' objects>),
 ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
 ('__gt__', <slot wrapper '__gt__' of 'object' objects>),
 ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
 ('__init__', <slot wrapper '__init__' of 'object' objects>),
 ('__init_subclass__', <function object.__init_subclass__>),
 ('__le__', <slot wrapper '__le__' of 'object' objects>),
 ('__lt__', <slot wrapper '__lt__' of 'object' objects>),
 ('__ne__', <slot wrapper '__ne__' of 'object' objects>),
 ('__new__', <function object.__new__(*args, **kwargs)>),
 ('__reduce__', <method '__reduce__' of 'object' objects>),
 ('__reduce_ex__', <method '__r

In [37]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']